# Multiclass Classification - PyTorch & NVIDIA DALI

## Plant Pathology 2021

### Check GPU Runtime

In [1]:
!nvidia-smi

Fri Apr 16 16:49:47 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.06    Driver Version: 450.51.06    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P0    33W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Installing NVIDIA-DALI

In [2]:
!pip install --extra-index-url https://developer.download.nvidia.com/compute/redist nvidia-dali-cuda110

Looking in indexes: https://pypi.org/simple, https://developer.download.nvidia.com/compute/redist


### Importing Required Libraries

In [3]:
from collections import defaultdict
import os
import shutil
import sys
import platform
import time

import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt

from nvidia.dali import pipeline_def, Pipeline
import nvidia.dali.fn as fn
import nvidia.dali.types as types
from nvidia.dali.plugin.pytorch import DALIClassificationIterator as PyTorchIterator
from nvidia.dali.plugin.pytorch import LastBatchPolicy

import torchvision.utils as utils
import torch
import torchvision
import skimage.transform
from skimage import io

### Creating OneHot for Training and Testing

In [4]:
data = pd.read_csv("../input/plant-pathology-2021-fgvc8/train.csv")
data

,image,labels
0,800113bb65efe69e.jpg,healthy
1,8002cb321f8bfcdf.jpg,scab frog_eye_leaf_spot complex
2,80070f7fb5e2ccaa.jpg,scab
3,80077517781fb94f.jpg,scab
4,800cbf0ff87721f8.jpg,complex
...,...,...
18627,fffb900a92289a33.jpg,healthy
18628,fffc488fa4c0e80c.jpg,scab
18629,fffc94e092a59086.jpg,rust
18630,fffe105cf6808292.jpg,scab frog_eye_leaf_spot


In [5]:
dct = defaultdict(list)

for i, label in enumerate(data.labels):
    for category in label.split():
        dct[category].append(i)
 
dct = {key: np.array(val) for key, val in dct.items()}
dct

{'healthy': array([    0,     5,     7, ..., 18626, 18627, 18631]),
 'scab': array([    1,     2,     3, ..., 18625, 18628, 18630]),
 'frog_eye_leaf_spot': array([    1,    14,    31, ..., 18612, 18619, 18630]),
 'complex': array([    1,     4,     8, ..., 18597, 18604, 18617]),
 'rust': array([    6,    21,    26, ..., 18601, 18616, 18629]),
 'powdery_mildew': array([   20,    39,    44, ..., 18532, 18617, 18618])}

In [6]:
new_df = pd.DataFrame(np.zeros((data.shape[0], len(dct.keys())), dtype=np.int8), columns=dct.keys())

for key, val in dct.items():
    new_df.loc[val, key] = 1

new_df.head()

,healthy,scab,frog_eye_leaf_spot,complex,rust,powdery_mildew
0,1,0,0,0,0,0
1,0,1,1,1,0,0
2,0,1,0,0,0,0
3,0,1,0,0,0,0
4,0,0,0,1,0,0


In [7]:
data = pd.concat([data, new_df], axis=1)
data.to_csv('better_train.csv', index = False)
data

,image,labels,healthy,scab,frog_eye_leaf_spot,complex,rust,powdery_mildew
0,800113bb65efe69e.jpg,healthy,1,0,0,0,0,0
1,8002cb321f8bfcdf.jpg,scab frog_eye_leaf_spot complex,0,1,1,1,0,0
2,80070f7fb5e2ccaa.jpg,scab,0,1,0,0,0,0
3,80077517781fb94f.jpg,scab,0,1,0,0,0,0
4,800cbf0ff87721f8.jpg,complex,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...
18627,fffb900a92289a33.jpg,healthy,1,0,0,0,0,0
18628,fffc488fa4c0e80c.jpg,scab,0,1,0,0,0,0
18629,fffc94e092a59086.jpg,rust,0,0,0,0,1,0
18630,fffe105cf6808292.jpg,scab frog_eye_leaf_spot,0,1,1,0,0,0


In [8]:
train_data = data.sample(frac=0.8).reset_index(drop=True)
train_data

,image,labels,healthy,scab,frog_eye_leaf_spot,complex,rust,powdery_mildew
0,ee4e84354b93d4c9.jpg,scab,0,1,0,0,0,0
1,a5c3aa145f345d4e.jpg,frog_eye_leaf_spot,0,0,1,0,0,0
2,ffd9a09d061e0e68.jpg,scab,0,1,0,0,0,0
3,809339deb2d6a50f.jpg,scab,0,1,0,0,0,0
4,961ff87d8c245ac2.jpg,frog_eye_leaf_spot,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...
14901,e49a079f4479f1a4.jpg,powdery_mildew,0,0,0,0,0,1
14902,b1f045999924adbe.jpg,rust,0,0,0,0,1,0
14903,d5928febfc2cc090.jpg,scab,0,1,0,0,0,0
14904,f69c896b31b14d61.jpg,rust,0,0,0,0,1,0


In [9]:
test_data = pd.merge(data,train_data, indicator=True, how='outer').query('_merge=="left_only"').drop('_merge', axis=1).reset_index(drop=True)
test_data

,image,labels,healthy,scab,frog_eye_leaf_spot,complex,rust,powdery_mildew
0,80077517781fb94f.jpg,scab,0,1,0,0,0,0
1,801d6dcd96e48ebc.jpg,healthy,1,0,0,0,0,0
2,8021b94d437eb7d3.jpg,healthy,1,0,0,0,0,0
3,80230a9a3f7a9f6b.jpg,scab,0,1,0,0,0,0
4,80261f473eafb92c.jpg,scab,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...
3721,fff00006af9e9c38.jpg,frog_eye_leaf_spot,0,0,1,0,0,0
3722,fff020893e070f27.jpg,healthy,1,0,0,0,0,0
3723,fff2e1c1e70300f0.jpg,frog_eye_leaf_spot,0,0,1,0,0,0
3724,fffb65761200b054.jpg,healthy,1,0,0,0,0,0


### Creating NVIDIA DALI PipeLine

In [10]:
class ExternalInputIterator(object):
    def __init__(self, batch_size, images_dir, data, device_id, num_gpus):
        self.images_dir = images_dir
        self.batch_size = batch_size
        self.data = data
        # whole data set size
        self.data_set_len = len(self.data)
        # based on the device_id and total number of GPUs - world size
        # get proper shard
        self.data = self.data[self.data_set_len * device_id // num_gpus:
                                self.data_set_len * (device_id + 1) // num_gpus]
        self.n = len(self.data)

    def __iter__(self):
        self.i = 0
        self.data = self.data.sample(frac=1).reset_index(drop=True)
        return self

    def __next__(self):
        batch = []
        labels = []

        if self.i >= self.n:
            self.__iter__()
            raise StopIteration

        for _ in range(self.batch_size):
            jpeg_filename, label = self.data.iloc[self.i % self.n].image, self.data.iloc[self.i % self.n][2:].to_numpy(dtype = np.uint8)
            jpeg_filename = os.path.join(self.images_dir, jpeg_filename)
            batch.append(torch.from_numpy(np.fromfile(jpeg_filename, dtype = np.uint8)))  # we can use numpy
            labels.append(torch.from_numpy(label))
            self.i += 1
        return (batch, labels)

    def __len__(self):
        return self.data_set_len

    next = __next__

In [11]:
def ExternalSourcePipelineTrain(batch_size, output_size, num_threads, device_id, external_data):
    pipe = Pipeline(batch_size, num_threads, device_id)
    with pipe:
        jpegs, labels = fn.external_source(source=external_data, num_outputs=2)
        images = fn.decoders.image(jpegs, device="mixed")
        images = fn.flip(images, horizontal = fn.random.coin_flip(), vertical = fn.random.coin_flip(), device = "gpu")
        images = fn.rotate(images, angle=fn.random.uniform(range=(-90., 90.)), device = "gpu")
        images = fn.random_resized_crop(images,
                size = output_size,
                random_area = [0.4, 1.0],
                random_aspect_ratio = [0.5, 1.5],
                device="gpu",
                )
        images = fn.crop_mirror_normalize(
                images,
                dtype=types.FLOAT,
                mean=[0.0, 0.0, 0.0],
                std=[255., 255., 255.],
                )
        pipe.set_outputs(images, labels)
    return pipe

In [12]:
def ExternalSourcePipelineTest(batch_size, output_size, num_threads, device_id, external_data):
    pipe = Pipeline(batch_size, num_threads, device_id)
    with pipe:
        jpegs, labels = fn.external_source(source=external_data, num_outputs=2)
        images = fn.decoders.image(jpegs, device="mixed")
        images = fn.resize(images, size=output_size)
        images = fn.crop_mirror_normalize(
                images,
                dtype=types.FLOAT,
                mean=[0.0, 0.0, 0.0],
                std=[255., 255., 255.],
                )
        pipe.set_outputs(images, labels)
    return pipe

### Checking PipeLine Validity

In [13]:
!mkdir Augmentation_Check

mkdir: cannot create directory ‘Augmentation_Check’: File exists


In [14]:
# eii = ExternalInputIterator(64, "../input/plant-pathology-2021-fgvc8/train_images", data, 0, 1)
# pipe = ExternalSourcePipelineTrain(batch_size=64, output_size=(224,224), num_threads=2, device_id = 0,
#                               external_data = eii)
# pii = PyTorchIterator(pipe, last_batch_padded=True, last_batch_policy=LastBatchPolicy.PARTIAL)

# count = 0
# for e in range(1):
#     for i, dt in enumerate(pii):
#         image = dt[0]['data']
#         if i < 10:
#             grid = utils.make_grid(image)
#             plt.imshow(grid.cpu().numpy().transpose((1, 2, 0)))
#             plt.savefig(f"./Augmentation_Check/{count}.png", dpi=300)
#             plt.close()
#             count+=1
#         sys.stdout.write(f"\r epoch: {e}, iter {i}, real batch size: {image.shape} and label {dt[0]['label'].shape}")
#     pii.reset()
# print("\nOK")
# torch.cuda.empty_cache()

### Model and Training Code

In [15]:
# Dictionary for pretrained models and their last layer name
pretrained_models = {
    "ResNet18": [torchvision.models.resnet18, "layer4", (224, 224)],
    "ResNet34": [torchvision.models.resnet34, "layer4", (224, 224)],
    "ResNet50": [torchvision.models.resnet50, "layer4", (224, 224)],
    "ResNet101": [torchvision.models.resnet101, "layer4", (224, 224)],
    "ResNet152": [torchvision.models.resnet152, "layer4", (224, 224)],
    "Alexnet": [torchvision.models.alexnet, "features", (256, 256)],
    "VGG11": [torchvision.models.vgg11_bn, "features", (224, 224)],
    "VGG13": [torchvision.models.vgg13_bn, "features", (224, 224)],
    "VGG16": [torchvision.models.vgg16_bn, "features", (224, 224)],
    "VGG19": [torchvision.models.vgg19_bn, "features", (224, 224)],
    "GoogleNet": [torchvision.models.googlenet, "inception5b", (224, 224)],
    "Inception": [torchvision.models.inception_v3, "Mixed_7c", (299, 299)],
}

In [16]:
# The main model object
class PlantPathology:
    """
    Model Architecture and Forward Training Path for the PlantPathology
    Idea is to use transfer Learning
    """

    def __init__(self, base_model="ResNet18", colab=False):
        assert base_model in [
            "ResNet18",
            "ResNet34",
            "ResNet50",
            "ResNet101",
            "ResNet152",
            "Alexnet",
            "VGG11",
            "VGG13",
            "VGG16",
            "VGG19",
            "GoogleNet",
            "Inception",
        ]

        # saving base model name to use it in saving the model
        self.base_model = base_model
        if colab:
            self.colab_training = f"drive/My Drive/{self.base_model}"
        else:
            self.colab_training = "."

        if not os.path.exists(f"{self.colab_training}/model"):
            os.mkdir(f"{self.colab_training}/model")
        if not os.path.exists(f"{self.colab_training}/model_results"):
            os.mkdir(f"{self.colab_training}/model_results")

        if os.path.exists(f"{self.colab_training}/model/ConvModel_{self.base_model}"):
            # check if the model is intialized before
            self.model = torch.load(
                f"{self.colab_training}/model/ConvModel_{self.base_model}"
            )
        else:
            # If not initialized before
            # Download it and save it
            self.model = pretrained_models[self.base_model][0](pretrained=True)
            for name, param in self.model.named_parameters():
                param.requires_grad = True

            # Modify last Fully Connected layer to predict for
            # Our requirements
            if self.base_model in ["Alexnet", "VGG11", "VGG13", "VGG16", "VGG19"]:
                num_ftrs = self.model.classifier[6].in_features
                self.model.classifier[6] = torch.nn.Linear(num_ftrs, 6)
            else:
                self.model.fc = torch.nn.Sequential(
                    torch.nn.Linear(self.model.fc.in_features, 500),
                    torch.nn.ReLU(),
                    torch.nn.Dropout(),
                    torch.nn.Linear(500, 6),
                )

            # Save model
            torch.save(
                self.model, f"{self.colab_training}/model/ConvModel_{self.base_model}"
            )

        # get final model for using it in Class Activation Map
        self.final_layer = self.model._modules.get(
            pretrained_models[self.base_model][1]
        )

        # Different image transformations for training, testing and displaying
        self.train_transformation = torchvision.transforms.Compose(
            [
                torchvision.transforms.RandomHorizontalFlip(),
                torchvision.transforms.RandomVerticalFlip(),
                torchvision.transforms.RandomRotation(90),
                torchvision.transforms.RandomResizedCrop(
                    pretrained_models[self.base_model][2],
                    scale=(0.4, 1.0),
                    ratio=(0.5, 1.5),
                    interpolation=2,
                ),
                torchvision.transforms.ToTensor(),
                torchvision.transforms.Normalize(
                    mean=[0,0,0], std=[255,255,255]
                ),
            ]
        )
        self.test_transformation = torchvision.transforms.Compose(
            [
                torchvision.transforms.Resize(pretrained_models[self.base_model][2]),
                torchvision.transforms.ToTensor(),
                torchvision.transforms.Normalize(
                    mean=[0,0,0], std=[255,255,255]
                ),
            ]
        )
        self.display_transformation = torchvision.transforms.Compose(
            [
                torchvision.transforms.Resize(pretrained_models[self.base_model][2]),
            ]
        )

    def train(
        self,
        optimizer,
        loss_fun,
        train_data,
        test_data,
        epochs=20,
        early_stopping_threshold=4,
        device="cuda",
        dali = False
    ):
        """
        Train function:
        parameters:
        optimizer   : optimizer object
        loss_fun    : Loss Function object
        train_data  : train dataloader
        test_data   : test  dataloader
        epochs      : default value 20
        early_stopping_threshold : Early stopping threshold
        device      : 'cuda' or 'cpu', default 'cuda'
        """

        # transfer model to device available
        self.model.to(device)

        max_accurracy = 0.0
        train_losses = []
        test_losses = []
        train_accuracies = []
        test_accuracies = []

        for epoch in range(epochs):
            start = time.time()

            training_loss = 0.0

            train_correct = 0
            train_total = 0

            # Training over batches
            self.model.train(mode=True)
            for i, train_batch in enumerate(train_data):
                d = train_batch[0]
                train_images, train_labels = d["data"], d["label"]
                # train_images = train_images.permute(0,3,1,2)
                train_images = train_images.to(device)
                train_labels = train_labels.to(device, dtype=torch.float32).squeeze()

                optimizer.zero_grad()
                train_output = self.model(train_images)

                if self.base_model == "Inception":
                    train_output = train_output.logits
#                 print(train_output.shape)
#                 print(train_labels.shape)
#                 print((torch.sigmoid(train_output) > 0.5).to(dtype=torch.float32))
#                 print(train_labels)
                train_loss = loss_fun(torch.sigmoid(train_output), train_labels)
                
                train_loss.backward()
                optimizer.step()

                training_loss += train_loss.item()
                
                train_predicted = (torch.sigmoid(train_output) > 0.5).to(dtype=torch.float32)
                train_total += train_labels.shape[0] * train_labels.shape[1]
                
#                 print("Equal: ", (train_predicted == train_labels).sum().item())
                
                train_ccount = (train_predicted == train_labels).sum().item()
                train_correct += train_ccount
                
                text = !nvidia-smi
                text = text[9][60:65]
                
                sys.stdout.write(
                    f"\rEpoch {epoch+1:03d}\t"
                    f"Train Loss => {train_loss:08.4f} "
                    f"Train Accuracy => "
                    f"{(train_ccount*100/(train_predicted.shape[0]*train_predicted.shape[1])):06.2f} "
                    f"GPU Utilization => {text}"
                )
        
            train_data.reset()

            training_accuracy = train_correct / train_total * 100

            valid_loss = 0.0
            test_correct = 0
            test_total = 0
            misses = 0
            previous_accuracy = 0
            tp = 0
            fp = 0
            fn = 0

            # Test over batches
            self.model.train(mode=False)
            with torch.no_grad():
                for i, test_batch in enumerate(test_data):    
                    d = test_batch[0]
                    test_images, test_labels = d["data"], d["label"]
                    # test_images = test_images.permute(0,3,1,2)
                    test_images = test_images.to(device)
                    test_labels = test_labels.to(device, dtype=torch.float32).squeeze()

                    test_output = self.model(test_images)

                    test_loss = loss_fun(torch.sigmoid(test_output), test_labels)
                    valid_loss += test_loss.item()

                    test_predicted = (torch.sigmoid(test_output) > 0.5).to(dtype=torch.float32)

                    test_total += test_labels.shape[0] * test_labels.shape[1]
        
                    test_ccount = (test_predicted == test_labels).sum().item()
                    
                    for i in range(test_labels.shape[0]):
                        for j in range(test_labels.shape[1]):
                            tp += int((test_labels[i][j] == 1) & ((test_predicted)[i][j] == 1))
                            fn += int((test_labels[i][j] != 0) & ((test_predicted)[i][j] == 0))
                            fp += int((test_labels[i][j] != 1) & ((test_predicted)[i][j] == 1))
                    test_correct += test_ccount

            testing_accuracy = test_correct / test_total * 100
            testing_precision = tp / (tp + fp) * 100
            testing_recall = tp / (tp + fn) * 100
            testing_f1 = (
                2.0
                * testing_recall
                * testing_precision
                / (testing_recall + testing_precision)
            )
            test_data.reset()

            sys.stdout.flush()
            sys.stdout.write("\r")

            time_taken = time.time() - start

            print(
                f"Epoch {epoch + 1:03d}\n"
                f"Train Loss => {training_loss:08.4f} "
                f"Train Accuracy => {training_accuracy:06.2f} "
                f"Test Loss => {valid_loss:08.4f} "
                f"Test Accuracy => {testing_accuracy:06.2f} "
                f"Test Precision => {testing_precision:06.2f}\n"
                f"Test Recall => {testing_recall:06.2f} "
                f"Test F1 Score => {testing_f1:06.2f} "
                f"Time Taken => {time_taken:08.4f}"
                f"\n{'='*100}"
            )

            train_losses.append(training_loss)
            test_losses.append(valid_loss)
            train_accuracies.append(training_accuracy)
            test_accuracies.append(testing_accuracy)

            # Save if it is better model than max_accuracy
            if testing_accuracy > max_accurracy:
                max_accurracy = testing_accuracy
                torch.save(
                    self.model,
                    f"{self.colab_training}/model/ConvModel_{self.base_model}",
                )

                with open(
                    f"{self.colab_training}/model_results/ConvModel_{self.base_model}_results.txt",
                    "w",
                ) as f:
                    f.writelines(
                        [
                            f"BaseModel: {self.base_model}\n",
                            f"Epochs: {epoch + 1:03d}\n",
                            f"Train Dataloader Batch Size: {train_data.batch_size}\n",
                            f"Test Dataloader Batch Size: {test_data.batch_size}\n",
                            f"Params for Optimizer: {optimizer.__repr__()}\n",
                            f"Train Loss: {training_loss:08.4f}\n",
                            f"Test Loss: {valid_loss:08.4f}\n",
                            f"Train Accuracy: {training_accuracy:06.2f}\n",
                            f"Test Accuracy: {testing_accuracy:06.2f}\n",
                            f"Test Precision: {testing_precision:06.2f}\n",
                            f"Test Recall: {testing_recall:06.2f}\n",
                            f"Test F1 Score: {testing_f1:06.2f}\n",
                            f"Time Taken: {time_taken:08.4f} seconds",
                        ]
                    )

            # Decide and stop early if needed
            if epoch >= 1:
                if (
                    previous_accuracy > testing_accuracy
                    and misses < early_stopping_threshold
                ):
                    misses += 1
                    previous_accuracy = testing_accuracy
                elif previous_accuracy > testing_accuracy:
                    print(f"Early Stopping....")
                    print(
                        f"Epoch {epoch + 1:03d}\t"
                        f"Train Loss => {training_loss:08.4f} "
                        f"Train accuracy => {training_accuracy:06.2f} "
                        f"Test Loss => {valid_loss:08.4f} "
                        f"Test Accuracy => {testing_accuracy:06.2f} "
                        f"Test Precision => {testing_precision:06.2f} "
                        f"Test Recall => {testing_recall:06.2f} "
                        f"Test F1 Score => {testing_f1:06.2f} "
                        f"Time Taken => {time_taken:08.4f}"
                    )
                    break
            previous_accuracy = testing_accuracy

            np.save(
                f"{self.colab_training}/model/train_losses_{self.base_model}",
                train_losses,
            )
            np.save(
                f"{self.colab_training}/model/train_accuracies_{self.base_model}",
                train_accuracies,
            )
            np.save(
                f"{self.colab_training}/model/test_losses_{self.base_model}",
                test_losses,
            )
            np.save(
                f"{self.colab_training}/model/test_accuracies_{self.base_model}",
                test_accuracies,
            )

    def test(self, loss_fun, test_data, device="cuda", has_labels=False, dali=False):
        print("Starting Evaluating....")
        start = time.time()
        valid_loss = 0.0
        test_correct = 0
        test_total = 0
        misses = 0
        previous_accuracy = 0
        tp = 0
        fp = 0
        fn = 0

        # Test over batches
        self.model.train(mode=False)
        with torch.no_grad():
            for i, test_batch in enumerate(test_data):    
                d = test_batch[0]
                test_images, test_labels = d["data"], d["label"]
                # test_images = test_images.permute(0,3,1,2)
                test_images = test_images.to(device)
                test_labels = test_labels.to(device, dtype=torch.float32).squeeze()

                test_output = self.model(test_images)

                test_loss = loss_fun(torch.sigmoid(test_output), test_labels)
                valid_loss += test_loss.item()

                test_predicted = (torch.sigmoid(test_output) > 0.5).to(dtype=torch.float32)

                test_total += test_labels.shape[0] * test_label.shape[1]

                test_ccount = (test_predicted == test_labels).sum().item()

                for i in test_labels.shape[0]:
                    for j in test_labels.shape[1]:
                        tp += int((test_labels[i][j] == 1) & ((test_predicted)[i][j] == 1))
                        fn += int((test_labels[i][j] != 0) & ((test_predicted)[i][j] == 0))
                        fp += int((test_labels[i][j] != 1) & ((test_predicted)[i][j] == 1))
                test_correct += test_ccount

        testing_accuracy = test_correct / test_total * 100
        testing_precision = tp / (tp + fp) * 100
        testing_recall = tp / (tp + fn) * 100
        testing_f1 = (
            2.0
            * testing_recall
            * testing_precision
            / (testing_recall + testing_precision)
        )
        test_data.reset()

        sys.stdout.flush()
        sys.stdout.write("\r")

    def CAM(self, image_path_input, overlay_path_output, device="cuda"):
        """
        CAM - Class Activation Map
        """

        # open image
        image = Image.open(image_path_input)
        image = image.convert("RGB")
        print(image.mode)

        tensor = self.test_transformation(image)

        prediction_var = torch.autograd.Variable(
            (tensor.unsqueeze(0)).cuda(), requires_grad=True
        )
        self.model.to(device)
        self.model.eval()

        class SaveFeatures:
            features = None

            def __init__(self, m):
                self.hook = m.register_forward_hook(self.hook_fn)

            def hook_fn(self, module, input, output):
                self.features = ((output.cpu()).data).numpy()

            def remove(self):
                self.hook.remove()

        activated_features = SaveFeatures(self.final_layer)
        prediction_var = prediction_var.to(device)
        prediction = self.model(prediction_var)

        pred_probabilities = torch.nn.functional.softmax(
            prediction, dim=0
        ).data.squeeze()

        activated_features.remove()

        torch.topk(pred_probabilities, 1)

        def getCAM(feature_conv, weight_fc, class_idx):
            _, nc, h, w = feature_conv.shape
            cam = weight_fc[class_idx].dot(feature_conv.reshape((nc, h * w)))
            cam = cam.reshape(h, w)
            cam = cam - np.min(cam)
            cam_img = cam / np.max(cam)
            return [cam_img]

        weight_softmax_params = list(self.model._modules.get("fc").parameters())
        weight_softmax = np.squeeze(weight_softmax_params[0].cpu().data.numpy())

        class_idx = torch.topk(pred_probabilities, 1)[1].int()

        overlay = getCAM(activated_features.features, weight_softmax, class_idx)

        plt.figure(figsize=(32, 15))
        plt.subplot(1, 2, 1)
        plt.imshow(self.display_transformation(image))
        plt.subplot(1, 2, 2)
        plt.imshow(self.display_transformation(image))
        plt.imshow(
            skimage.transform.resize(overlay[0], tensor.shape[1:3]),
            alpha=0.4,
            cmap="jet",
        )
        plt.savefig(overlay_path_output)

In [17]:
if torch.cuda.is_available():
    print("Using GPU")
    device = torch.device("cuda")
else:
    print("Using CPU")
    device = torch.device("cpu")

Using GPU


### Hyperparameters

In [18]:
batch_size = 32
learning_rate = 0.0001
dali = True
N_GPUS = 1
epochs = 10
base_model = "ResNet50"
optimizer_name = 'Adam'
colab = False
image_dir = "../input/plant-pathology-2021-fgvc8/train_images"

In [19]:
print("Creating Model Object: ")
model = PlantPathology(base_model, colab=colab)

Creating Model Object: 


### DataLoader

In [20]:
train_eii = ExternalInputIterator(batch_size, image_dir, train_data, 0, 1)
pipe_train = ExternalSourcePipelineTrain(batch_size=batch_size, output_size=pretrained_models[base_model][2], num_threads=2, device_id = 0,
                              external_data = train_eii)
train_data_loader = PyTorchIterator(pipe_train, last_batch_padded=True, last_batch_policy=LastBatchPolicy.PARTIAL)

test_eii = ExternalInputIterator(batch_size, image_dir, test_data, 0, 1)
pipe_test = ExternalSourcePipelineTest(batch_size=batch_size, output_size=pretrained_models[base_model][2], num_threads=2, device_id = 0,
                              external_data = test_eii)
test_data_loader = PyTorchIterator(pipe_test, last_batch_padded=True, last_batch_policy=LastBatchPolicy.PARTIAL)

### Optimizer and Training

In [21]:
optimizers = {
    "Adam": torch.optim.Adam,
    "SGD": torch.optim.SGD,
    "RMSprop": torch.optim.RMSprop,
    "Adagrad": torch.optim.Adagrad,
    "Adadelta": torch.optim.Adadelta,
}

In [22]:
optimizer = optimizers[optimizer_name](
    model.model.parameters(), lr=learning_rate
)

print("Starting Training")    
model.train(
    optimizer,
    torch.nn.BCELoss(),
    train_data_loader,
    test_data_loader,
    epochs=epochs,
    device=device,
    dali=dali
)
print("Completed Training")

Starting Training
Epoch 001	Train Loss => 000.0696 Train Accuracy => 097.40 GPU Utilization =>  45% 
Train Loss => 060.1852 Train Accuracy => 095.04 Test Loss => 012.1428 Test Accuracy => 095.84 Test Precision => 091.28
Test Recall => 085.24 Test F1 Score => 088.16 Time Taken => 483.1266
Epoch 002	Train Loss => 000.0764 Train Accuracy => 096.35 GPU Utilization =>  58% 
Train Loss => 055.1978 Train Accuracy => 095.38 Test Loss => 012.3646 Test Accuracy => 095.81 Test Precision => 090.40
Test Recall => 086.08 Test F1 Score => 088.18 Time Taken => 505.4285
Epoch 003	Train Loss => 000.1588 Train Accuracy => 093.23 GPU Utilization =>  77% 
Train Loss => 050.5900 Train Accuracy => 095.76 Test Loss => 010.9844 Test Accuracy => 096.27 Test Precision => 091.18
Test Recall => 087.96 Test F1 Score => 089.54 Time Taken => 509.2396
Epoch 004	Train Loss => 000.1879 Train Accuracy => 094.27 GPU Utilization =>  68% 
Train Loss => 047.7581 Train Accuracy => 096.01 Test Loss => 011.1908 Test Accuracy =>